In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.mode.chained_assignment = None 

In [ ]:
import pandas as pd

data_frame=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")



In [ ]:
data_frame.head(15)

In [ ]:
positive_values = data_frame[(data_frame.review .notnull()) & (data_frame.sentiment == "positive")]

print(positive_values.head(15))

In [ ]:
text_1=positive_values.review[0]
text_1


**The world cloud for the positive terms**

In [ ]:
from wordcloud import WordCloud,STOPWORDS 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
wordcloud = WordCloud(width=700,height=400, max_font_size=80, max_words=100, background_color="white").generate(text_1)
f = plt.figure() 
f.set_figwidth(15) 
f.set_figheight(10) 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
negative_values=data_frame[(data_frame.review .notnull()) & (data_frame.sentiment == "negative")]

print(negative_values.head(15))

In [ ]:
text_2=negative_values.review[11]

text_2

**The world cloud for the Negative terms**

In [ ]:

wordcloud = WordCloud(width=700,height=400, max_font_size=80, max_words=100, background_color="white").generate(text_2)


f = plt.figure() 
f.set_figwidth(15) 
f.set_figheight(10) 

 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,3))
data_frame.sentiment.value_counts().plot(kind='bar', rot=360)
plt.show()

Summary & describtion of the dataset :

In [ ]:
data_frame['sentiment'].value_counts()

In [ ]:
data_frame.describe()

 spliting data into train and test set by using sklearn’s train_test_split
    

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_frame,test_size=0.25)

In [ ]:
print(train.shape)
print(test.shape)

#0.25*50000=12500
#0.75*50000=37500


**Text preprocessing phase **

1.Turn sentiment into categorical value

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train['sentiment'] = labelencoder.fit_transform(train['sentiment'])
test['sentiment'] = labelencoder.fit_transform(test['sentiment'])



In [ ]:
test['sentiment'].head(10)

 2. Remove none text and special character

In [ ]:
import re
import string
pattern = re.compile(r'<br\s*/><br\s*/>>*|(\-)|(\\)|(\/)')
def preprocess_reviews(reviews):
    reviews = [pattern.sub(" ",item) for item in reviews]
    return reviews
train_clean = preprocess_reviews(train['review'])
test_clean = preprocess_reviews(test['review'])
train['review'] = train_clean
test['review'] = test_clean
def remove_punctuation(input):
    table = str.maketrans('','',string.punctuation)
    return input.translate(table)
train['review'] = train['review'].apply(remove_punctuation)
test['review'] = test['review'].apply(remove_punctuation)




In [ ]:
train['review']

 3. Convert all text to lowercase

In [ ]:
train['review'] = train['review'].str.lower()
test['review'] = test['review'].str.lower()

In [ ]:
train['review']

 4. Remove line breaks

In [ ]:
def remove_linebreaks(input):
    text = re.compile(r'\n')
    return text.sub(r' ',input)
train['review'] = train['review'].apply(remove_linebreaks)
test['review'] = test['review'].apply(remove_linebreaks)

In [ ]:
train['review']

5. Tokenization

In [ ]:

from nltk.tokenize import word_tokenize
train['review'] = train['review'].apply(word_tokenize)
test['review'] = test['review'].apply(word_tokenize)

In [ ]:
train['review']

6. Remove stopword

In [ ]:
%%time

from nltk.corpus import stopwords
def remove_stopwords(input1):
    words = []
    for word in input1:
        if word not in stopwords.words('english'):
            words.append(word)
    return words
train['review'] = train['review'].apply(remove_stopwords)
test['review'] = test['review'].apply(remove_stopwords)

In [ ]:
train['review']

7. Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
train['review'] = train['review'].apply(lemma_wordnet)
test['review'] = test['review'].apply(lemma_wordnet)

In [ ]:
train['review']

8. Combine individual words

In [ ]:
def combine_text(input):
    combined = ' '.join(input)
    return combined
train['review'] = train['review'].apply(combine_text)
test['review'] = test['review'].apply(combine_text)

In [ ]:
train['review']

**TF-IDF**

In [ ]:

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model,model_selection

vectorizer = feature_extraction.text.TfidfVectorizer(norm = None)
vectorizer.fit(train)

X_train_tfidf = vectorizer.fit_transform(train['review'])
X_test_tfidf = train['sentiment']
Y_train_tdidf =vectorizer.transform(test['review'])


 

Rigde with TF-IDF

In [ ]:

alpha = [500.0, 1500.0, 2500.0, 3000.0 ,3500.0]
for a in alpha:
    ridge = linear_model.RidgeClassifier(a)
    scores = model_selection.cross_val_score(ridge, X_train_tfidf, X_test_tfidf, cv=5, scoring='f1')
    print("alpha: ",a)
    print(scores)
    print(np.mean(scores))
    print('\n')

MultinomialNB with TF-IDF

In [ ]:

from sklearn.naive_bayes import MultinomialNB
alpha = [175.0, 200.0, 225.0, 250.0, 300.0]
for a in alpha:
    mnb = MultinomialNB(a)
    scores = model_selection.cross_val_score(mnb, X_train_tfidf, X_test_tfidf, cv=5, scoring='f1')
    print('alpha: ', a)
    print(scores)
    print(np.mean(scores))
    print('\n')

**Hashing**

In [ ]:

from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer()
hv.fit(train)
X_train_hash = hv.fit_transform(train['review'])
X_test_hash = train['sentiment']
Y_train_hash = hv.transform(test['review'])




Rigde with Hash

In [ ]:

from sklearn import linear_model,model_selection
alpha = [1.1, 1.2, 1.3, 1.4, 1.5, 2.0]
for a in alpha:
    ridge = linear_model.RidgeClassifier(a)
    scores = model_selection.cross_val_score(ridge, X_train_hash, X_test_hash, cv=5, scoring='f1')
    print("alpha: ",a)
    print(scores)
    print(np.mean(scores))
    print('\n')


**Bag of words**

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(train)
X_train_bow = cv.fit_transform(train['review'])
X_test_bow = train['sentiment']
Y_train_bow = cv.transform(test['review'])
Y_test = test['sentiment']

Rigde with bag of word

In [ ]:
from sklearn import model_selection  
from sklearn import linear_model
alpha = [80.0, 90.0, 100.0, 110.0, 120.0]
for a in alpha:
    ridge = linear_model.RidgeClassifier(a)
    scores = model_selection.cross_val_score(ridge, X_train_bow, X_test_bow, cv=5, scoring='f1')
    print("alpha: ",a)
    print(scores)
    print(np.mean(scores))
    print('\n')

MultinomialNB with bag of word

In [ ]:
alpha = [1e-10, 1e-5, 0.1, 1.0, 2.0]
for a in alpha:
    mnb = MultinomialNB(a)
    scores = model_selection.cross_val_score(mnb, X_train_bow, X_test_bow, cv=5, scoring='f1')
    print('alpha: ', a)
    print(scores)
    print(np.mean(scores))
    print('\n')

In [ ]:
from sklearn.metrics import accuracy_score
ridge = linear_model.RidgeClassifier(1.4)
ridge.fit(X_train_hash, X_test_hash)
test['sentiment_pred'] = ridge.predict(Y_train_hash)
y_true = test['sentiment']
y_pred = test['sentiment_pred']
accuracy_score(y_true, y_pred)
acc=accuracy_score(y_true, y_pred)
print("Accuricy score is :  ",acc )




In [ ]:
from sklearn.metrics import classification_report,confusion_matrix 
print(classification_report(y_true, y_pred, target_names = ['Bad Reviews','Good Reviews']))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true, y_pred)


print(cm)

In [ ]:

plt.figure(figsize = (8,8))
sns.heatmap(cm,cmap= "Blues", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
d = {"Review":test['review'],'sentiment':test['sentiment'],'predected sentiment':test['sentiment_pred']}
import pandas as pd
from sklearn.utils import shuffle
df = pd.DataFrame(d)
df.head(15)